In [1]:
import numpy as np
from math import *
from numba import njit

from matplotlib import pyplot as plt
%matplotlib notebook

### Physical  constants

First and foremost some general physical constants are defined and scaled to astronomical size. These constants are used in later equations. 

We have the gravitational constant `G` which is in units of $\text{AU}^{3} \text{solarmasses}^{-1} \text{years}^{-2}$. Next we have one solar mass `M` in the units of kilograms.

In [2]:
G = 39.47970417034628
M = 1.98855e30 

###  Structure of the System

We start by determining the number of bodies `num_bodies` that will be in the system. This is done by executing the first command `np.int(np.random.uniform())` which will return a random integer between 6 and 9. Next we define the number of dimensions the system will have and set that to three.

--------------

For our groups we need to define some values. This means that one needs to define a number of arrays of zeroes for respectively the position, velocity, acceleration, force and mass. This is to place our simulated data into.

In [3]:
num_bodies = 3
dimension  = 3

# Main Arrays
pos   = np.zeros((num_bodies, dimension))
vel   = np.zeros((num_bodies, dimension))
acc   = np.zeros((num_bodies, dimension))
force = np.zeros((num_bodies, dimension))

mass     = np.zeros((num_bodies))
init_vel = np.zeros((num_bodies))
rp       = np.zeros((num_bodies))
ra       = np.zeros((num_bodies))
a        = np.zeros((num_bodies))
e        = np.zeros((num_bodies))

# Arrays for plotting
inpos = np.zeros((num_bodies, dimension))
invel = np.zeros((num_bodies, dimension))

### Plot the initial Positions

The code below Plots the initial positions that we just defined in the previous cell.

In [4]:
# Planetary Masses 
mass[0] = 1.98855e30/M       # Sun
mass[1] = 1.8982e27/M       # Jupiter
mass[2] = 5.6834e26/M       # Saturn

# Planetary Distances 
# Jupiter 
rp[1] = 4.9501
ra[1] = 5.4588
a[1] = (rp[1]+ra[1])/2

# Saturn
rp[2] = 9.0412
ra[2] = 10.1238
a[2] = (rp[2]+ra[2])/2
    


# Orbital Eccentricities
e[1] = 0.0489              # Jupiter 
e[2] = 0.0565              # Saturn

# Initial Velocities
#for m in range(1,num_bodies):
#    init_vel[m] = np.sqrt(G*mass[0]*((2*rp[m])/(ra[m]*(rp[m]+ra[m]))))

# Initial Velocities
#for m in range(1,num_bodies):
 #   init_vel[m] = np.sqrt(G*mass[0]*((2/a[m])-(1/r_a[m])))
    
# Initial Velocities
for m in range(1,num_bodies):
    init_vel[m] = np.sqrt(((G*mass[0])/(a[m]))*((1+e[m])/(1-e[m])))
    

pos[:,1] = a
vel[:,0] = init_vel

# Velocity of the Center of Mass
CM_vel_x = np.sum((vel[:,0]*mass))/np.sum(mass)
vel[:,0] = vel[:,0]-CM_vel_x

print('masses')
print(mass)
print('initial positions')
print(pos)
print('initial velocities')
print(vel)

masses
[1.00000000e+00 9.54564884e-04 2.85806241e-04]
initial positions
[[0.      0.      0.     ]
 [0.      5.20445 0.     ]
 [0.      9.5825  0.     ]]
initial velocities
[[-0.00337065  0.          0.        ]
 [ 2.88899687  0.          0.        ]
 [ 2.14451585  0.          0.        ]]


### Force Function

Next we have to define the force each body has on every other body in the system. To do this we define a function `grav_force` which has elements of position, mass and force. We define `num_bodies` and `D` to be the magnitudes of `pos`'s dimentions.

We make a loop where for the i'th element in `num_bodies` we define a three dimentional array of zeros. Then we can calculate the force on the i'th element from every other element. This is done by a loop where we look at the j'th element in `num_bodies` for which is not equal to i. From here it is possible to calculate `force[i]` from Newtons law of Universal Gravitation.

In [5]:
@njit
def grav_force(pos, mass, force): 
    num_bodies, D = pos.shape 
    for i in np.arange(num_bodies): 
        force[i] = np.zeros(D) 
        
        for j in np.arange(num_bodies):
            if j!=i:
                r_vec = pos[i]-pos[j]
                r_mag = np.linalg.norm(r_vec)
                r_hat = r_vec/r_mag
                force[i] += (-G*mass[i]*mass[j])/(r_mag**2)*r_hat

#### Leapfrog Function

The next order of buisness is to define the Leapfrog function.

In [6]:
@njit
def leapfrog(pos, mass, vel, force, sim_time, dt):
    num_steps = ceil(sim_time/dt) # one this line we discretize time, I think
    num_bodies, D = pos.shape
    
    #---------we write up the initial arrays of zeroes to put in values later----------
    times = np.zeros((num_steps))
    positions = np.zeros((num_steps, num_bodies, D))
    velocities = np.zeros((num_steps, num_bodies, D))
    accelerations = np.zeros((num_steps, num_bodies, D))
    time = 0 # we start by setting the time to zero
    
    for step in range(num_steps):
        time += dt
        grav_force(pos, mass, force)
        acc = (force.T/mass).T
        vel += acc*dt
        pos += vel*dt
        
        times[step] = time 
        positions[step] = pos
        velocities[step] = vel
        accelerations[step] = acc
    
    return times, positions, velocities, accelerations

In [ ]:
times, positions, velocities, accelerations = leapfrog(pos, mass, vel, acc, 10, 1e-4)

%time _ = leapfrog(pos, mass, vel, acc, 10, 1e-4)

In [ ]:
print('num_bodies:',num_bodies)

plt.figure(figsize=(6, 6))
ax3 = plt.subplot(111)
ax3.plot(positions[:,:,0],positions[:,:,1],'-')
ax3.set_title('Oribits of Jupiter and Saturn')
plt.legend(['Sun', "Jupiter", 'Saturn'], loc='upper right')

plt.xlabel("AU")
plt.ylabel("AU")

positions.shape

In [9]:
JSpos = np.array([positions[:,0,0],positions[:,0,1],positions[:,1,0],positions[:,1,1]]) #,positions[:,2,0],positions[:,2,1]
JSpos = JSpos.T

JSvel = np.array([velocities[:,0,0],velocities[:,0,1],velocities[:,1,0],velocities[:,1,1]]) #,velocities[:,2,0],velocities[:,2,1]
JSvel = JSvel.T

JSacc = np.array([accelerations[:,0,0],accelerations[:,0,1],accelerations[:,1,0],accelerations[:,1,1]]) #,accelerations[:,2,0],accelerations[:,2,1]
JSacc = JSacc.T


np.save('JSpos.npy', JSpos)
np.save('JSvel.npy', JSvel)
np.save('JSacc.npy', JSacc)

print(JSpos)

[[-3.37065098e-07  1.51421035e-11  2.88899687e-04  5.20444999e+00]
 [-6.74130195e-07  4.54263104e-11  5.77799374e-04  5.20444996e+00]
 [-1.01119529e-06  9.08526208e-11  8.66699058e-04  5.20444991e+00]
 ...
 [ 2.31821989e-03  1.11734368e-02 -5.49955961e+00 -2.28622793e+00]
 [ 2.31830667e-03  1.11732559e-02 -5.49963408e+00 -2.28598568e+00]
 [ 2.31839344e-03  1.11730750e-02 -5.49970853e+00 -2.28574342e+00]]
